# This script processes the o2par results (tbb version of qmin) now with indices to point to the cloud and parallel tree

1. Compile o2par with:
```
make bin/o2par
```
2. Run the benchmarking script
```
cd scripts
python3 run_par_optim2.py
```
3. The output that we will process is `o2_partree_<hostname>.txt` that should be already saved in the `Results` folder.


In [ ]:
import os
# get the hostname of the server
hostname = os.popen("hostname").read().strip()
# ensure the directory exists
os.makedirs(hostname, exist_ok=True)
# ensure the file with baseline results exists
o2_file = f'o2_partree_{hostname}.txt'
# if it is not in Results
if not os.path.exists(o2_file):
    # is the file already in the directory?
    assert os.path.exists(os.path.join(hostname, o2_file)), f'File {o2_file} not found: something went wrong with the baseline benchmark.'
# if it is in Results
else:
    # copy the file to the directory
    assert os.system(f'mv {o2_file} {hostname}/') == 0, f'Failed to move {o2_file} to {hostname}/'
# rename o2_file
o2_file = os.path.join(hostname, o2_file)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sys
from statistics import mean
from common.utils import get_nprocs

cloud_list = ['AlcoyH','ArzuaH','BrionFH','BrionUH']

# number of threads used in the execution
num_threads = get_nprocs()

def tokenize1(filename,noH=False):
    experiment ={}

    with open(filename) as f:
        for line in f:
            tokens = line.split()
            if "Running:" in tokens:
                if noH:
                    name=tokens[2].split("/")[3][:-1]
                else:
                    name=tokens[2].split("/")[3]
                nth=int(tokens[6])
                if name not in experiment:
                    experiment[name]={}
            if 'Quadtree' in tokens:
                experiment[name][nth]=[float(tokens[5])]
            if "STAGE" in tokens:
                experiment[name][nth].append(float(tokens[5]))
            if 'Average:' in tokens:
                experiment[name][nth].append(float(tokens[1]))

    #print(experiment)

    results = {}

    for i in experiment:
        results[i]={}
        for j in experiment[i]:
            results[i][j]={}
            results[i][j]['qtree']=experiment[i][j][0]
            results[i][j]['stage1']=mean(experiment[i][j][1:16:3])
            results[i][j]['stage2']=mean(experiment[i][j][2:16:3])
            results[i][j]['stage3']=mean(experiment[i][j][3:16:3])
            results[i][j]['owm']=experiment[i][j][16]
            results[i][j]['total']=experiment[i][j][0]+experiment[i][j][16]
    return results

def tokenize2(filename,noH=False):
    experiment ={}

    with open(filename) as f:
        for line in f:
            tokens = line.split()
            if "Running:" in tokens:
                if noH:
                    name=tokens[3].split("/")[3][:-1]
                else:
                    name=tokens[3].split("/")[3]
                nth=int(tokens[11])
                level=int(tokens[19])
                if name not in experiment:
                    experiment[name]={}
                if level not in experiment[name]:
                    experiment[name][level]={}
                if nth not in experiment[name][level]:
                    experiment[name][level][nth]=[]
            if "STAGE" in tokens:
                experiment[name][level][nth].append(float(tokens[5]))
            if 'Quadtree' in tokens:
                experiment[name][level][nth].append(float(tokens[5]))
            if 'Average' in tokens:
                experiment[name][level][nth].append(float(tokens[2]))

    #print(experiment)

    results = {}

    for i in experiment:
        results[i]={}
        for j in experiment[i]:
            results[i][j]={}
            for k in experiment[i][j]:
                results[i][j][k]={}
                results[i][j][k]['qtree']=experiment[i][j][k][15]
                results[i][j][k]['stage1']=mean(experiment[i][j][k][0:15:3])
                results[i][j][k]['stage2']=mean(experiment[i][j][k][1:15:3])
                results[i][j][k]['stage3']=mean(experiment[i][j][k][2:15:3])
                results[i][j][k]['owm']=experiment[i][j][k][16]
                results[i][j][k]['total']=experiment[i][j][k][15]+experiment[i][j][k][16]
    return results

res1=tokenize1(os.path.join(hostname, f'o1_qtree_{hostname}.txt'))
# res2=tokenize1("o2_parallel_coffee.txt",True) # optim2 without parallel tree construction
res4=tokenize2(os.path.join(hostname, f'o2_partree_{hostname}.txt'))  # optim2 with parallel tree construction


In [ ]:
def plot_levels(nth,results):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    #fig = plt.figure()
    labels=['OWM Trav.','Tree Const.','Total']
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    for i,name in enumerate(cloud_list):
        x=list(results[name].keys())
        axs[i].plot(np.array(x), np.array([results[name][j][nth]['owm'] for j in x]), marks[0], linewidth=linew, markersize=markers)
        axs[i].plot(np.array(x), np.array([results[name][j][nth]['qtree'] for j in x]), marks[1], linewidth=linew, markersize=markers)
        axs[i].plot(np.array(x), np.array([results[name][j][nth]['qtree']+results[name][j][nth]['owm'] for j in x]), marks[2], linewidth=linew, markersize=markers)

        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('Level', fontsize=xlabelfs)
        axs[i].set_xticks(x)
        axs[i].tick_params(axis='x', labelsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()
    if(nth==1):
        fig.suptitle('Execution time (sec.)',  fontweight='bold', fontsize=18)
    if(nth>1):
        fig.suptitle(f'{nth}-threads execution time (sec.) O2 {hostname.upper()}',  fontweight='bold', fontsize=18)

    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    axs[0].legend(labels,loc='best', fontsize= 14)
    pp = PdfPages(os.path.join(hostname, f'Optim2_{nth}coresExecTime-{hostname}.pdf'))
    pp.savefig(fig)
    pp.close()

# plot 8 and 16 if AlderLake
if hostname == 'alder':
    plot_levels(8, res4)
    plot_levels(16, res4)
else:
    plot_levels(num_threads[-1], res4)


In [ ]:
def plot_times_index(results, maxth=8):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    #fig = plt.figure()
    labels=['OWM Trav.','Tree Const.','Total']
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    for i,name in zip(range(4), cloud_list):
        # select the best level for each experiment
        bestlevel=min(results[name], key=lambda x: results[name][x][maxth]['total'])
        x=list(results[name][bestlevel].keys())
        axs[i].plot(np.array(x), np.array([results[name][bestlevel][nth]['owm'] for nth in x]), marks[0], linewidth=linew, markersize=markers)
        axs[i].plot(np.array(x), np.array([results[name][bestlevel][nth]['qtree'] for nth in x]), marks[1], linewidth=linew, markersize=markers)
        axs[i].plot(np.array(x), np.array([results[name][bestlevel][nth]['total'] for nth in x]), marks[2], linewidth=linew, markersize=markers)

        axs[i].set_title(name+ " (lev="+str(bestlevel)+")",fontsize=16)
        axs[i].set_xlabel('Num threads', fontsize=xlabelfs)
        axs[i].set_xticks(x)
        axs[i].tick_params(axis='x', labelsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()

    fig.suptitle(f'Execution time (sec.) O2 {hostname.upper()}',  fontweight='bold', fontsize=18)
    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    axs[0].legend(labels,loc='best', fontsize= 14)
    #axs[i].show()

plot_times_index(res4, maxth=num_threads[-1])

# Compare OWM traversal improvement

In [ ]:
def printowm(nth,res):#best level
    for i in res:
        bestlevel=min(res[i], key=lambda x: res[i][x][nth]['total'])
        print("Cloud {} has best level = {}.".format(i, bestlevel))
        print("Cloud {} has {}-cores OMP OWM-o1 quadtree time = {:.2f}.".format(i,nth, res1[i][nth]['owm']))    
        print("Cloud {} has {}-cores OWM-o2 partree time = {:.2f}. Speedup (o2 partree vs o1) = {:.2f}x.".format(i,nth,res[i][bestlevel][nth]['owm'], res1[i][nth]['owm']/res[i][bestlevel][nth]['owm'] ))    

#printowm(1,res4)
printowm(num_threads[-1],res4)

# Compare Tree construction improvement

In [ ]:
def printtree(nth,res):#best level
    for i in res:
        bestlevel=min(res[i], key=lambda x: res[i][x][nth]['total'])
        print("Cloud {} has best level = {}.".format(i, bestlevel))
        print("Cloud {} has {}-cores OMP TreeConst-o1 quadtree time = {:.2f}.".format(i,nth, res1[i][nth]['qtree']))    
        print("Cloud {} has {}-cores TreeConst-o2 partree time = {:.2f}. Speedup (o2 partree vs o1) = {:.2f}x.".format(i,nth,res[i][bestlevel][nth]['qtree'], res1[i][nth]['qtree']/res[i][bestlevel][nth]['qtree'] ))    

#printtree(1,res4)
printtree(num_threads[-1], res4)

# Compare Total (OWM traversal + Tree construction) improvement

In [ ]:
def printtotal(nth,res):#best level
    for i in res:
        bestlevel=min(res[i], key=lambda x: res[i][x][nth]['total'])
        print("Cloud {} has best level = {}.".format(i, bestlevel))
        print("Cloud {} has {}-cores OMP Total time = {:.2f}.".format(i,nth, res1[i][nth]['total']))    
        print("Cloud {} has {}-cores Total time = {:.2f}. Speedup (o2 partree vs o1) = {:.2f}x.".format(i,nth,res[i][bestlevel][nth]['total'], res1[i][nth]['total']/res[i][bestlevel][nth]['total'] ))    

#printtotal(1,res4)
printtotal(num_threads[-1], res4)  

# Plot Optimization 1 Quadtree execution time (for reference)

In [ ]:
def plot_alltimes(results):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    labels=['OWM Trav.','Tree Const.','Total']
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    for i,name in zip(range(4), cloud_list):
        x=list(results[name].keys())
        axs[i].plot(np.array(x), np.array([results[name][nth]['owm'] for nth in x]), marks[0], linewidth=linew, markersize=markers)
        axs[i].plot(np.array(x), np.array([results[name][nth]['qtree'] for nth in x]), marks[1], linewidth=linew, markersize=markers)
        axs[i].plot(np.array(x), np.array([results[name][nth]['qtree']+results[name][nth]['owm'] for nth in x]), marks[2], linewidth=linew, markersize=markers)

        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('Num threads', fontsize=xlabelfs)
        axs[i].set_xticks(x[::2])
        axs[i].tick_params(axis='x', labelsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()

    fig.suptitle('Execution time (sec.)',  fontweight='bold', fontsize=18)
    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    axs[0].legend(labels,loc='best', fontsize= 14)
    #axs[i].show()

plot_alltimes(res1)


In [ ]:
#Configuration variables
def plot_res(res, maxth=8):
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8

    fig = plt.figure()

    marks=['o-','x-','s-','v-','+-']
    bestlevel=[min(res[i], key=lambda x: res[i][x][maxth]['total']) for i in res]
    numthreads=list(res[cloud_list[0]][bestlevel[0]].keys())
    labels=['Tree Const.','OWM Trav.','Total']

    fig, axs = plt.subplots(nrows=1, ncols=3,figsize=(15, 5), constrained_layout=True)
    sizes=[i+" (lev="+str(j)+")" for i,j in zip(res,bestlevel)]
    sizes.append('Ideal')

    for phase,subfig in zip(['qtree','owm','total'],range(3)):
        print("Phase: "+phase+ " speedup for each cloud and best level for each cloud: "+str(list(res.keys())))
        for (i,z,lev) in zip(res,marks,bestlevel):
            print(np.array([res[i][lev][1][phase]/res[i][lev][j][phase] for j in numthreads]))
            axs[subfig].plot(np.array(numthreads), np.array([res[i][lev][1][phase]/res[i][lev][j][phase] for j in numthreads]), z, linewidth=linew, markersize=markers)
        axs[subfig].plot(np.array(numthreads), np.array(numthreads), '-', linewidth=linew, markersize=markers)
        axs[subfig].set_title('Speedup '+labels[subfig],  fontweight='bold', fontsize=titlefs)
        axs[subfig].set_xlabel('Number of cores', fontsize=xlabelfs)
        axs[subfig].set_xticks(numthreads)
        axs[subfig].tick_params(axis='x', labelsize=xticksfs)
        axs[subfig].grid()

    axs[0].set_ylabel('Speedup', fontsize=ylabelfs)
    axs[0].legend(sizes,loc='best', fontsize= legendfs)
#    plt.yticks(fontsize=yticksfs)

    plt.show()
    return fig

fig = plot_res(res4, num_threads[-1])

In [ ]:
pp = PdfPages(os.path.join(hostname, f'Speedup-o2-partree-{hostname}.pdf'))
pp.savefig(fig)
pp.close()

# Save data to All_Optimizations.csv

In [ ]:
output = os.path.join(hostname, f'All_Optimizations-{hostname}.csv')

f = open(output, "a")
# select the max number of threads
maxth = num_threads[-1]
for i in res4:
    bestlevel=min(res4[i], key=lambda x: res4[i][x][maxth]['total'])
    print("Opt2-Par; {}; {:.5f}; {:.5f};{};{}".format(i,res4[i][bestlevel][maxth]['qtree'],res4[i][bestlevel][maxth]['owm'],bestlevel,0))
    f.write("Opt2-Par;{};{:.5f};{:.5f};{};{}\n".format(i,res4[i][bestlevel][maxth]['qtree'],res4[i][bestlevel][maxth]['owm'],bestlevel,0))

f.close()